In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/contactinfo_txt", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/header_json", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/address_xlsx", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/detail1_csv", True)

Out[1]: True

In [0]:
%python
detail1_csv=spark.read.format("csv").option("inferschema",True).option("header",True).load("dbfs:/FileStore/tables/Detail-1.csv")
contactinfo_txt=spark.read.format("csv").option("inferschema",True).option("header",True).option("sep","\t").load("dbfs:/FileStore/tables/contactinfo.txt")
header_json=spark.read.format("json").option("header",True).option("inferschema",True).option("multiline",True).load("dbfs:/FileStore/tables/header.json")
address_xlsx=spark.read.format("com.crealytics.spark.excel").option("header",True).option("inferschema",True).option("sep",",").load("dbfs:/FileStore/tables/Address.xlsx")

In [0]:
%python
detail1_csv.write.mode("overwrite").format("parquet").saveAsTable("detail1_csv")
contactinfo_txt.write.mode("overwrite").format("parquet").saveAsTable("contactinfo_txt")
header_json.write.mode("overwrite").format("parquet").saveAsTable("header_json")
address_xlsx.write.mode("overwrite").format("parquet").saveAsTable("address_xlsx")

In [0]:
%sql
SELECT * FROM detail1_csv LIMIT 5

id,first_name,middle_name,last_name,email,ssn,gender,religion,marital_status,date_of_birth,deceased_date,spoken_language_1,spoken_language_2,company,job_role,job_hiredate
70001,Hettie,null,Keenlayside,jkeenlayside0@disqus.com,168-92-1075,F,Buddhism,Widowed,1939-08-05,null,West Frisian,Swahili,Gabcube,Clinical Specialist,1964-01-29
70002,Reade,null,Laverenz,dlaverenz1@senate.gov,782-24-9907,M,Christianity,Widowed,1941-05-14,null,Swati,Danish,Skibox,Staff Scientist,1958-05-18
70003,Minnnie,null,Baack,dbaack2@sina.com.cn,726-01-1271,F,Buddhism,Married,1982-11-20,null,Swati,null,Dabjam,Paralegal,2011-06-10
70004,Tana,Agata,Aiken,aaiken3@nydailynews.com,492-62-0968,F,null,null,1929-02-18,null,New Zealand Sign Language,Punjabi,Aimbu,VP Marketing,2014-10-08
70005,Cyndia,null,Tolomelli,ltolomelli4@istockphoto.com,802-24-1062,F,null,null,1920-05-31,null,null,Albanian,Edgepulse,Senior Developer,1931-01-16


In [0]:
%sql
SELECT * FROM header_json LIMIT 5

id,insurer_id,relationship
70001,40184,child
70002,40092,friend
70003,40233,spouse
70004,40058,spouse
70005,40088,friend


In [0]:
%sql
SELECT * FROM address_xlsx LIMIT 5

id,city,issued_date,address_line_1,address_line_2,state,address_type,zipcode
70376.0,Edwardburgh,2015-07-18T00:00:00.000+0000,2302 Shannon Wall,Apt. 419,Missouri,Residental,64801
70376.0,Lake Diana,2021-01-21T00:00:00.000+0000,842 David Grove Suite 069,Apt. 283,South Carolina,Mail,62440
70377.0,New Kellyview,2021-01-09T00:00:00.000+0000,02737 Crawford Common,Suite 436,Washington,Mail,35780
70377.0,East Robertview,2010-08-14T00:00:00.000+0000,37464 Delgado Forest Suite 919,Apt. 633,Iowa,Residental,36908
70378.0,West Catherine,2018-03-03T00:00:00.000+0000,3221 Michael Harbors,Suite 447,Florida,Residental,49167


In [0]:
%sql
SELECT * FROM contactinfo_txt LIMIT 5

id,phone,usage_type
70001,(455) 3130004,Work
70002,(994) 4561640,Work
70003,(771) 6498755,Work
70004,(450) 8886723,Work
70005,(423) 1700133,Work


In [0]:
%sql
CREATE OR REPLACE TABLE nova_project_table AS
(
  SELECT
    h.id AS source_id,
    h.insurer_id AS subscriber_id,
    d.first_name,
    d.middle_name,
    d.last_name,
    CASE
      WHEN gender = 'M' THEN 'Mr.'
      WHEN gender = 'F' AND d.marital_status in ('Married','Widowed') THEN 'Mrs.'
      WHEN gender = 'F' AND d.marital_status in ('Single','Divorced') THEN 'Ms.'
      WHEN gender = 'F' AND d.marital_status is null THEN 'Ms.'
      ELSE null
    END AS prefix_name,
    CASE
      WHEN d.first_name = d.middle_name AND h.relationship = 'child' THEN 'Jr.'
      WHEN job_role LIKE '%Engineer%' THEN 'Er'
      WHEN job_role LIKE '%Analyst%' THEN 'Analyst'
      WHEN job_role LIKE '%Manager%' THEN 'Mgr'
      WHEN job_role LIKE '%Administrator%' THEN 'Admin'
      WHEN job_role LIKE '%Developer%' THEN 'Dev'
      WHEN job_role LIKE '%Assistant%' THEN 'Asst'
      WHEN job_role LIKE '%Technician%' THEN 'Tech'
      WHEN job_role LIKE '%Account%' THEN 'Acct'
      ELSE null END AS suffix_name,
    (COALESCE(prefix_name,'') || COALESCE(d.first_name,'') ||' '|| COALESCE(d.middle_name,'') ||' '|| COALESCE(d.last_name,'')) AS name,
    'Nova_health' AS record_source,
    CURRENT_TIMESTAMP() AS record_created_ts,
    True AS is_verified,
    ARRAY_AGG(STRUCT(
      a.address_type,
      a.address_line_1,
      a.address_line_2,
      a.city,
      a.state AS state_province,
      (CASE WHEN LEN(zipcode)>5 THEN LEFT(zipcode,5)
            WHEN LEN(zipcode)=5 THEN zipcode END) AS zip_code_extension,
      (CASE WHEN LEN(zipcode)>5 THEN RIGHT(zipcode,4)
            WHEN LEN(zipcode)=4 THEN zipcode END) AS postal_code,
      'United States of America' AS country)) AS addresses,
    ARRAY_AGG(DISTINCT STRUCT(c.usage_type AS phone_type , c.phone AS number)) AS phones,
    CASE WHEN regexp_like(d.email,'.*@*\.*') then d.email else 'Invalid mail' END AS email,
    False AS privacy_preference,
    ssn AS national_id,
    d.gender,
    d.marital_status,
    d.date_of_birth,
    YEAR(d.date_of_birth) AS year_of_birth,
    CASE WHEN d.deceased_date IS NULL THEN false ELSE true END AS deceased_ind,
    YEAR(TO_DATE(d.deceased_date,'M/d/yyyy')) - YEAR(d.date_of_birth) AS deceased_age,
    TO_DATE(d.deceased_date,'M/d/yyyy') AS deceased_date,
    CASE
      WHEN spoken_language_1 IS NOT null AND spoken_language_2 IS NOT null THEN ARRAY(spoken_language_1,spoken_language_2)
      WHEN spoken_language_1 IS null AND spoken_language_2 IS NOT null THEN ARRAY(spoken_language_2)
      WHEN spoken_language_1 IS NOT null AND spoken_language_2 IS null THEN ARRAY(spoken_language_1)
      WHEN spoken_language_1 == spoken_language_2 THEN ARRAY(spoken_language_1)
      ELSE null
    END AS languages,
    ARRAY(STRUCT(
      d.company AS employer_name,
      d.job_role AS employee_role,
      CASE WHEN d.deceased_date IS NOT null THEN 'Inactive' ELSE 'Active' END AS employment_status,
      d.job_hiredate AS employee_hiredate)) AS Employment,
    map('relationship', h.relationship,'religion', d.religion) AS additional_source_value 
  FROM
    header_json h
  LEFT JOIN
    detail1_csv d ON h.id = d.id
  LEFT JOIN
    contactinfo_txt c ON h.id = c.id
  LEFT JOIN
    address_xlsx a ON h.id = a.id
  GROUP BY
    h.id,
    h.insurer_id,
    d.first_name,
    d.middle_name,
    d.last_name,
    d.email,
    d.gender,
    d.marital_status,
    d.date_of_birth,
    d.deceased_date,
    d.spoken_language_1,
    d.spoken_language_2,
    d.company,
    d.ssn,
    d.job_role,
    d.job_hiredate,
    h.relationship,
    d.religion
);

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM nova_project_table LIMIT 20

source_id,subscriber_id,first_name,middle_name,last_name,prefix_name,suffix_name,name,record_source,record_created_ts,is_verified,addresses,phones,email,privacy_preference,national_id,gender,marital_status,date_of_birth,year_of_birth,deceased_ind,deceased_age,deceased_date,languages,Employment,additional_source_value
70001,40184,Hettie,null,Keenlayside,Mrs.,null,Mrs.Hettie Keenlayside,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Mail, 4307 Ashley Village Suite 758, null, New Kyle, North Dakota, 87337, null, United States of America), List(Residental, 046 Cox Lights, Suite 600, Griffinhaven, South Carolina, 71709, null, United States of America))","List(List(Work, (455) 3130004))",jkeenlayside0@disqus.com,false,168-92-1075,F,Widowed,1939-08-05,1939,false,null,null,"List(West Frisian, Swahili)","List(List(Gabcube, Clinical Specialist, Active, 1964-01-29))","Map(relationship -> child, religion -> Buddhism)"
70002,40092,Reade,null,Laverenz,Mr.,null,Mr.Reade Laverenz,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Residental, 737 Banks Row, Apt. 505, North Heather, Arkansas, null, 2341, United States of America), List(Mail, 183 Dalton Viaduct, Suite 844, South Natalie, Oregon, 77714, null, United States of America))","List(List(Work, (994) 4561640))",dlaverenz1@senate.gov,false,782-24-9907,M,Widowed,1941-05-14,1941,false,null,null,"List(Swati, Danish)","List(List(Skibox, Staff Scientist, Active, 1958-05-18))","Map(relationship -> friend, religion -> Christianity)"
70003,40233,Minnnie,null,Baack,Mrs.,null,Mrs.Minnnie Baack,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Mail, 807 Jesus Mills Suite 598, Suite 735, Churchbury, Texas, 97223, null, United States of America), List(Residental, 27634 Miller Prairie, null, West Ruth, North Dakota, 73151, null, United States of America))","List(List(Work, (771) 6498755))",dbaack2@sina.com.cn,false,726-01-1271,F,Married,1982-11-20,1982,false,null,null,List(Swati),"List(List(Dabjam, Paralegal, Active, 2011-06-10))","Map(relationship -> spouse, religion -> Buddhism)"
70004,40058,Tana,Agata,Aiken,Ms.,null,Ms.Tana Agata Aiken,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Residental, 9831 Robert Falls, Apt. 086, Michelleland, Oregon, 05921, 5281, United States of America), List(Mail, 36717 Philip Common, Suite 278, Thomasborough, Idaho, 91582, 4725, United States of America))","List(List(Work, (450) 8886723))",aaiken3@nydailynews.com,false,492-62-0968,F,null,1929-02-18,1929,false,null,null,"List(New Zealand Sign Language, Punjabi)","List(List(Aimbu, VP Marketing, Active, 2014-10-08))","Map(relationship -> spouse, religion -> null)"
70005,40088,Cyndia,null,Tolomelli,Ms.,null,Ms.Cyndia Tolomelli,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Mail, 95855 Davis Lodge, Suite 059, Kimberlymouth, Louisiana, 33733, null, United States of America), List(Residental, 0861 Caldwell Dam, Suite 783, North Robertborough, New Mexico, 31718, null, United States of America))","List(List(Work, (423) 1700133))",ltolomelli4@istockphoto.com,false,802-24-1062,F,null,1920-05-31,1920,false,null,null,List(Albanian),"List(List(Edgepulse, Senior Developer, Active, 1931-01-16))","Map(relationship -> friend, religion -> null)"
70006,40170,Johnny,Renaud,Gibben,Mr.,null,Mr.Johnny Renaud Gibben,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Residental, 8737 Flores Extension Suite 549, null, Jasonbury, Idaho, 20277, null, United States of America), List(Mail, 1049 Riggs Stream Suite 632, Suite 465, New Christopher, Maine, 43342, null, United States of America))","List(List(Work, (334) 1254061))",rgibben5@tumblr.com,false,563-98-1576,M,Single,1958-07-01,1958,false,null,null,List(Georgian),"List(List(Oodoo, Human Resources Assistant I, Active, 2021-12-26))","Map(relationship -> child, religion -> Buddhism)"
70007,40194,Judas,null,Mitford,Mr.,null,Mr.Judas Mitford,nova_health,2024-02-01T10:43:17.553+0000,false,"List(List(Residental, 7475 Michael Land, Suite 392, New L